In [1]:
#conda install -c plotly plotly=4.0

In [2]:
import plotly.graph_objects as go
import plotly
import pandas as pd
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'plotly'

In [ ]:
#State Coords
# File to Load 
state_coords_to_load = "input/state_cords.csv"

# Read the firearms data
coord_data = pd.read_csv(state_coords_to_load)
coord_data.head()

In [ ]:
# File to Load 
fireams_data_to_load = "input/nics-firearm-background-checks.csv"

# Read the firearms data
firearms_data = pd.read_csv(fireams_data_to_load)
firearms_data.head()

In [ ]:
#Sort 2017
firearms_2017 = firearms_data[firearms_data['month'].str.contains('2017')]

#Registration sum 2017 & Set Index
registration_sum = firearms_2017.groupby(['state'])['totals'].sum()
registration_sum = registration_sum.reset_index()

#Hand Gun sum 2017 & Set Index
state_handguns_sum = firearms_2017.groupby(['state'])['handgun'].sum()
state_handguns_sum = state_handguns_sum.reset_index()

#Long Gun sum 2017 & Set Index
state_longguns_sum = firearms_2017.groupby(['state'])['long_gun'].sum()
state_longguns_sum = state_longguns_sum.reset_index()

#Permit sum 2017 & Set Index
permit_sum = firearms_2017.groupby(['state'])['permit'].sum()
permit_sum = permit_sum.reset_index()

data_pair = pd.merge(registration_sum, state_handguns_sum, how="left", on=["state", "state"])
other_pair = pd.merge(permit_sum, state_longguns_sum, how="left", on=["state", "state"])
totals_df = pd.merge(data_pair, other_pair, how="left", on=["state", "state"])

#data_pair = pd.merge(registration_sum, state_handguns_sum, how="left", on=["state", "state"])
#totals_df = pd.merge(data_pair, state_longguns_sum, how="left", on=["state", "state"])

totals_df.head()

In [ ]:
#World Atlas.COM
# File to Load 
state_population_to_load = "input/state_populations.csv"

# Read the firearms data
population_data = pd.read_csv(state_population_to_load)
population_data.head()

In [ ]:
# Combine the data into a single dataset with a left merge on "state"
data_pair = pd.merge(totals_df, population_data, how="left", on=["state", "state"])
df = pd.merge(data_pair, coord_data, how="left", on=["state", "state"])
df.head()

In [ ]:
for col in df.columns:
    df[col] = df[col].astype(str)

df['text'] = df['state'] + '<br>' + \
    'Total Handgun Registrations ' + df['handgun'] + '<br>' + \
    'Population ' + df['population'] + '<br>' + \
    'Populations Rank ' + df['pop_rank']
    
plt = go.Figure(data=go.Choropleth(
    #state digraph = location code
    locations=df['code'],
    z=df['handgun'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=df['text'], # hover text
    marker_line_color='black', # line markers between states
    colorbar_title="Hundreds of Thousands"
))

plt.update_layout(
    title_text='2017 Handgun Registrations Per State<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
#plt.savefig('output/handguns2017.png', bbox_inches="tight")
plt.show()

In [ ]:
for col in df.columns:
    df[col] = df[col].astype(str)

df['text'] = df['state'] + '<br>' + \
    'Total Long Gun Registrations ' + df['long_gun'] + '<br>' + \
    'Population ' + df['population'] + '<br>' + \
    'Populations Rank ' + df['pop_rank']
    
plt = go.Figure(data=go.Choropleth(
    #state digraph = location code
    locations=df['code'],
    z=df['long_gun'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=df['text'], # hover text
    marker_line_color='black', # line markers between states
    colorbar_title="Hundreds of Thousands"
))

plt.update_layout(
    title_text='2017 Long Gun Registrations Per State<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
#plt.savefig('output/longguns2017.png', bbox_inches="tight")
#plt.write_image('output/longguns2017.png')
plt.show()

In [ ]:
for col in df.columns:
    df[col] = df[col].astype(str)

df['text'] = df['state'] + '<br>' + \
    'Total Permits ' + df['permit'] + '<br>' + \
    'Population ' + df['population'] + '<br>' + \
    'Populations Rank ' + df['pop_rank']
    
plt = go.Figure(data=go.Choropleth(
    #state digraph = location code
    locations=df['code'],
    z=df['permit'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=df['text'], # hover text
    marker_line_color='black', # line markers between states
    colorbar_title="Millions"
))

plt.update_layout(
    title_text='2017 Permits Per State<br>Note: Kentucky performs monthly checks on concealed carry permit holders<br>(Hover for breakdown)(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
#plt.savefig('output/permits2017.png', bbox_inches="tight")
plt.show()

In [ ]:
for col in df.columns:
    df[col] = df[col].astype(str)

df['text'] = df['state'] + '<br>' + \
    'Total Firearms Transaction' + df['totals'] + '<br>' + \
    'Population ' + df['population'] + '<br>' + \
    'Populations Rank ' + df['pop_rank']
    
plt = go.Figure(data=go.Choropleth(
    #state digraph = location code
    locations=df['code'],
    z=df['totals'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=df['text'], # hover text
    marker_line_color='black', # line markers between states
    colorbar_title="Millions"
))

plt.update_layout(
    title_text='2017 Firearm Transactions Per State<br>Note: Kentucky performs monthly checks on concealed carry permit holders <br>(Hover for breakdown)',
    #title_text=('Note: Kentucky performs monthly checks on Concealed Permit Holders'),
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)
#plt.savefig('output/transactions2017.png', bbox_inches="tight")
#plt.savefig('output/transactions2017.png')
plt.show()